# Examen TP - Analyse de données

### Instructions

Dans cette séance :
- Vous allez travailler en binôme.
- Vous devez remplir les portions de code indiquées, et en fin de séance déposer votre programme sur l'espace dédié de la page Moodle, rénommé comme nom1_nom2.ipynb. **Attention à ce que les noms des deux membres du binômes soient bien indiqués dans le nom de votre fichier**.
- Vous pourrez consulter vos programmes réalisés lors des séances précédentes, ainsi que les transparents de cours et corrigés fournis.
- Vous pourrez également consulter la documentation de python sur internet, et des sources pédagogiques standard comme Wikipedia.
- En revanche, tout plagiat de code trouvé en ligne ou partagé entre différent binômes est strictement interdit et passible de sanction disciplinaire.
- L'utilisation des logiciels de communication (mail, messagerie instantanée, etc) n'est pas autorisée.


### Concernant le TP

- Nous allons travailler avec des histogrammes issus de distributions gaussiennes, et nous allons effectuer une ACP dans l'espace des densités de probabilité. En particulier, nous allons utiliser des approximations par Bsplines.
- Tout au long du TP, vous aurez des questions auxquelles il faudra répondre directement sur le notebook.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

Nous allons en premier lieu créer un jeu de données exemple ('toy dataset') de $n$ histogrammes définis sur une grille de points $x$.

In [ ]:
n = 50
x = np.arange(-200,200)


**Exercice:** Créer un jeu de données gaussiennes sur lequel nous effectuerons l'ACP de densité de probabilités. Pour cela:
- Générer aléatoirement $n$ moyennes $m_i$ de loi uniforme dans $[-120,120]$, et $n$ variances $\sigma_i^2$ de loi uniforme dans $[8,32]$.
- Générer $n$ histogrammes de gaussiennes $\mathcal{N}(m_i,\sigma_i^2)$ définis sur $x$ que vous stockerez dans une matrice ```data``` de taille ```n*len(x)```. SI nécessaire, s'aider du package ```scipy.stats```.
- Afficher l'ensemble des données sur un graphe.

**Remarque:** Les histogrammes doivent sommer à 1.

In [ ]:
...

**Exercice:**
- Effectuer une ACP sur les données.
- Afficher les projections des données sur les deux premières composantes principales (comme effectué en TP) dans l'ordre croissant de leur score en utilisant le dégradé de couleur défini par la variable ```map1 = matplotlib.colormaps['winter'].resampled(...)```. Afficher également le vecteur moyen.

In [ ]:
...

**Question:**
- Que pouvez-vous en déduire sur la variation globale de votre jeu de données?
- Quelles sont vos critiques sur cette méthode?

In [ ]:
...

# ACP dans l'espace des densités de probabilité

Pour cela nous devons en premier lieu transformer chaque histogramme de prénom $h_i, i=1,\ldots,n$ via l'application $T$ suivante:

$$T(h_i)= F_i^{-1}\circ \bar{F_n}-\text{id},$$

où
- $F_i^{-1}$ est la fonction quantile de l'histogramme $h_i$, c'est-à-dire la fonction inverse de la fonction de répartition
- $\bar{F_n}$ est défini comme l'inverse de la fonction quantile $\bar{F_n}^{-1}$ telle que:
$$\bar{F_n}^{-1}=\frac{1}{n} \sum_{i=1}^{n} \ F_i^{-1}.$$

**Exercice:** Construire une fonction "cdf_quant" qui prend en entrée un histogramme $h$, la grille de points ```x```, et un vecteur ```t ```, et qui renvoie ses fonctions de répartition $F$ et quantile $F^{-1}$ évaluée en ```t```. La fonction quantile doit être approcher par Bsplines de degré $1$, sans régularisation.

In [ ]:


def cdf_quant(h, x, t):
    
    ## Afin de pouvoir inverser la fonction de répartition, il faut qu'elle soit strictement croissante. Nous
    # considérons donc un support x_new sur lequel l'histogramme h_new est strictement plus grand que epsilon.
    epsilon = 1e-14
    seuil = h>epsilon
    x_new = ...
    h_new = ...
    
    ## Calcul de la fonction de répartition
    cdf = ...
    cdf_new = ...
    
    ## Calcul de la fonction quantile dans la variable 'quant'
    ...
    
    ## On fixe les bornes de la fonction quantile 'quant' afin d'être bien défini sur x
    quant[0] = x[0]
    quant[-1] = x[-1]
    
    return cdf, quant
    
    

**Exercice:**
- Calculer $\bar{F_n}^{-1}$ sur une grille ```t = np.linspace(0,1,100000)```, dans la variable ```Fn_bar_quant```, ainsi que sa fonction inverse $\bar{F_n}$ dans la variable ```Fn_bar``` sur ```x``` (par approximation Bsplines, ou bien par une autre fonction d'interpolation de python).
- Afficher le graphe de ```Fn_bar_quant```et ```Fn_bar``` dans deux plots différents.

In [ ]:

...


La densité $\bar{f}_,$ (dans la variable ```fn_bar```) associée à la fonction de répartition $\bar{F}_n$ est alors récupérée en dérivant la fonction de répartition $\bar{F}_n$. Il s'agit de la moyenne aux sens des densités de probabilité.

In [ ]:
f_bar = Fn_bar.copy()
f_bar[1:] = Fn_bar[1:] - Fn_bar[:-1]
f_bar[0] = f_bar[1]

plt.plot(x,data.T)
plt.plot(x,f_bar, linewidth = 3, color = 'black')
plt.title('Données et barycentre')
plt.show()

**Exercice:** Construire une fonction ```map_T``` qui prend en entrée une fonction quantile $F_i^{-1}$ et $\bar{F_n}$, ainsi que les vecteurs ```t``` et ```x``` et qui renvoie l'application

$$T(h_i)= F_i^{-1}\circ \bar{F_n}-\text{id},$$

évaluée en ```x```. Pour cela, utiliser l'approximation par Bsplines de degré 1, ou bien une autre fonction d'interpolation de python.

In [ ]:
...

**Exercice:** Créer une matrice $V$ de taille ```n*len(x)``` qui contient l'ensemble des applications $T$ pour les ```n``` histogrammes $h$.

In [ ]:
...

## ACP sur les vecteurs de $V$, dans l'espace de Hilbert $L^2(\mathbb{R})$ **pondérée**  par $f_{bar}$ de norme $\Vert\cdot\Vert_{f_{bar}}$définie par

$$\Vert h \Vert_{f_{bar}}^2 = \int |h(x)|^2 f_{bar}(x)dx.$$

**Exercice:**
- Calculer les deux premiers vecteurs propres $u_1, u_2$ de la matrice de covariance de $V$. **REMARQUE IMPORTANTE!** L'espérance de la covariance est prise par rapport au produit scalaire pondéré par $f_{bar}$, i.e.
$$\mathbb{E}(X_iX_j) = \langle X_i,X_j\rangle_{f_{bar}} = \sum_k X_i^{k}X_j^{k}f_{bar}^k.$$
Dans le code ci-après, $V_{pond}$ a été pondéré au préalable, il suffit donc de calculer une covariance classique sur ```V_pond```.
- Normaliser les vecteurs $u_1$ et $u_2$ afin que $\Vert u_1 \Vert_{f_{bar}}^2=\Vert u_2 \Vert_{f_{bar}}^2=1$. **ATTENTION!** $f_{bar}$ contient des valeurs nulles sur son support.

In [ ]:
V_mean = np.mean(V,0)
## Pondération du produit scalaire par f_bar
V_pond = np.dot(V-V_mean,np.diag(np.sqrt(f_bar)))

...

**Exercice:**
- Créer une matrice ```proj1``` de taille ```n*len(x)``` contenant les projections des histogrammes sur la 1ère  composantes auxquelles on ajoute la fonction identité. Autrement dit:

$$ \text{proj}_1[i,:] = \text{id} + V_{\text{mean}} + \langle h_i-V_{mean},u_1\rangle_{f_{bar}} * u_1,$$

où $u_1$ est le vecteur propre associé à la plus grande valeur propre de la matrice de covariance de $V$.
- Faire la même chose dans une matrice ```proj2``` pour le 2ème vecteur propre $u_2$.

**ATTENTION!** Le produit scalaire $\langle \cdot,\cdot\rangle_{f_{bar}}$ est pondéré par $f_{bar}$ !

In [ ]:
...

Enfin, afin de revenir dans l'espace des mesures de probabilité, nous allons utiliser la fonction ```pushforward_density```qui est donnée ci-après.

In [ ]:
from scipy.signal import savgol_filter

def consecutive(vect, stepsize=1):
    return np.split(vect, np.cumsum( np.where(vect[1:] - vect[:-1] > 1) )+1)

def pushforward_density(T, bar, x):
    epsilon = 1e-5
    inter = x[1]-x[0]

    cst = np.where(np.abs(T[0:-1]-T[1:])<epsilon)[0]
    aa = consecutive(cst)

    noncst = np.arange(len(x))
    noncst = np.delete(noncst,cst)
    bb = consecutive(noncst)

    dR = -savgol_filter(T[noncst],15,1, deriv = 1)/inter

    knots, coeff, k = splrep(T[noncst], bar[noncst]/np.abs(dR), k = 1)
    B = BSpline(knots, coeff,k, extrapolate = False)
    g = B(x[noncst])
    
    h = np.zeros(len(x))
    h[noncst] = g

    return h

**Exercice:** Afficher les projections des données sur les deux premières composantes principales $g_1$ et $g_2$ définies par:

```g1[i] = pushforward_density(proj1[i,:], f_bar, x)```

(et de même pour $g_2$), dans l'ordre croissant de leur score ```proj1``` en utilisant le dégradé de couleur défini par la variable ```map1```.

In [ ]:
...

**Exercice:** Commenter ces deux plots : que représentent-ils? Quelles sont les différences avec l'ACP du début de TP? Cette ACP est-elle plus pertinente ou non? Expliquer pourquoi.

In [ ]:
...

**Exercice bonus:** Expliquer ce que fait la fonction ```pushforward_density```.

In [ ]:
...